In [ ]:
import matplotlib.pyplot as plt
import torch
import time
import psutil

In [ ]:
def predict_and_visualize_custom_image(model, image_tensor, show=True):
    with torch.no_grad():
        predicted_image = model(image_tensor)
    if show:
        image_tensor = image_tensor.cpu()
        predicted_image = predicted_image.cpu()
        fig, axes = plt.subplots(1, 2, figsize=(8, 4))

        axes[0].imshow(image_tensor.squeeze(0).squeeze(0), cmap="gray")
        axes[0].set_title("Input Image")
        axes[0].axis("off")

        axes[1].imshow(predicted_image.squeeze(0).squeeze(0), cmap="gray")
        axes[1].set_title("Predicted Rotated Image")
        axes[1].axis("off")

        plt.tight_layout()
        plt.show()

    return predicted_image


def measure_average_performance(model, image_tensor, num_runs=1000):
    execution_times = []

    for _ in range(num_runs):
        start_time = time.time()
        predict_and_visualize_custom_image(model, image_tensor, show=False)
        end_time = time.time()
        execution_times.append(end_time - start_time)

    average_time = sum(execution_times) / num_runs
    print(f"Average Execution Time over {num_runs} runs: {average_time:.10f} seconds")

    process = psutil.Process()
    memory_info = process.memory_info()
    print(f"Memory Usage: {memory_info.rss / (1024 * 1024):.2f} MB")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

custom_image_tensor = torch.zeros(1, 1, 11, 11)
custom_image_tensor[:, :, 3, 4] = 1.0
custom_image_tensor[:, :, 4, 4] = 1.0
custom_image_tensor[:, :, 5, 2:9] = 1.0
custom_image_tensor = custom_image_tensor.to(device)


model = torch.jit.load("../weights/best_model.pt")
model.eval().to(device)


predict_and_visualize_custom_image(model, custom_image_tensor, show=True)
measure_average_performance(model, custom_image_tensor)